In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def box3d(n):
    d_x = d_y = d_z = 1 / n
    x0 = y0 = z0 = -0.5
    x = np.arange(x0, -x0, d_x, dtype=float)
    y = np.arange(y0, -y0, d_y, dtype=float)
    z = np.arange(z0, -z0, d_z, dtype=float)
    x = np.append(x,0.5)
    y = np.append(y,0.5)
    z = np.append(z,0.5)
    cube = np.stack(np.meshgrid(x, y, z))
    # print(cube)
    # one = abs(cube[0,:,:,:]) == 0.5
    # two = abs(cube[1,:,:,:]) == 0.5
    # thr = abs(cube[2,:,:,:]) == 0.5
    # w = np.logical_or(one,two)
    # w = np.logical_or(w,thr)
    # points = cube[:,w].reshape(cube[:,w].shape[1], 3)
    # return points # just sides of the cube
    return cube.reshape(3,-1) # cube
    
Q = box3d(10)
ax = plt.axes(projection='3d')
Q = np.swapaxes(Q,0,1)
Q = Q[(abs(Q) == 0.5).sum(axis=1) >= 2]
# ax.scatter3D(Q[0,:],Q[1,:],Q[2,:])
ones = np.ones(Q.shape[0]).reshape(Q.shape[0],1)
Q = np.concatenate((Q,ones),axis=1)
ax.scatter3D(Q[:,0],Q[:,1],Q[:,2])

In [ ]:
def projectpoints(K, R, t, Q):
    Q.reshape((Q.shape[1],Q.shape[0]))
    T = np.concatenate((R,t),axis=1)
    # ones = np.ones(Q.shape[0]).reshape(Q.shape[0],1)
    # Q = np.concatenate((Q,ones),axis=1)
    ppsx = []
    ppsy = []
    for i in range(Q.shape[0]):
        p = Q[i,:]
        projected = K @ T @ p.reshape(4,1)
        ppsx.append(float(projected[0]/projected[2]))
        ppsy.append(float(projected[1]/projected[2]))
    return ppsx,ppsy

K = R = np.eye(3)
theta = np.pi / 6
R = np.array([
    [np.cos(theta), 0, np.sin(theta)],
    [0, 1, 0],
    [-np.sin(theta), 0, np.cos(theta)],
])
t = np.array([[0,0,4]]).T
(ppsx,ppsy) = projectpoints(K,R,t,Q)
plt.scatter(ppsx, ppsy) # 2D image